In [0]:
import numpy as np
import tensorflow as tf
import tensorflow.keras.layers as layers

In [3]:
(train_x, train_y), (test_x, test_y) = tf.keras.datasets.cifar10.load_data()

170500096/170498071 [==============================] - 11s 0us/step


In [0]:
def ResNetUnit(x, filter_size, kernel_size, bn, act, strides=1):
  if bn:
    x = layers.BatchNormalization()(x)
  if act:
    x = layers.Activation('relu')(x)
  x = layers.Conv2D(filter_size, kernel_size, strides, 'same', kernel_initializer='he_normal')(x)
  return x

def ResNet(input_shape, block_num, num_classes):
  # 32, 32, 16
  # stage 0 32, 32, 64
  # stage 1 16, 16, 128
  # stage 2 8, 8, 256
  filter_in = 16
  filter_out = 64
  inputs = tf.keras.layers.Input(input_shape)
  x = layers.Conv2D(filter_in, 3, 1, 'same')(inputs)
  for stage in range(3):
    for block in range(block_num):
      strides = 1
      if block == 0 and stage != 0:
        strides = 2   #down sample
      y = ResNetUnit(x, filter_in, 1, True, True, strides)
      y = ResNetUnit(y, filter_in, 3, True, True)
      y = ResNetUnit(y, filter_out, 1, True, True)
      if block == 0:
        x = ResNetUnit(x, filter_out, 1, False, False, strides)    #match size of x to y
      x = layers.add([x, y])
      filter_in = filter_out
    filter_out = filter_in * 2
  x = layers.BatchNormalization()(x)
  x = layers.Activation('relu')(x)
  x = layers.AveragePooling2D(pool_size=8)(x)
  x = layers.Flatten()(x)
  outputs = layers.Dense(num_classes, activation='softmax', kernel_initializer='he_normal')(x)
  return tf.keras.models.Model(inputs=inputs, outputs=outputs)

In [0]:
epochs = 100
input_shape = train_x.shape[1:]
block_num = (110-2)//9   #9 layers in block loop (bn, act, conv(1)), (bn, act, conv(3)), (bn, act, conv(1))
num_classes = 10
train_step = len(train_x)//32
test_step = len(test_x)//32

In [0]:
model = ResNet(input_shape,block_num, num_classes)

In [0]:
model.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(0.01),
    metrics=['accuracy']
)

In [0]:
train_y_one = tf.keras.utils.to_categorical(train_y,num_classes)
test_y_one = tf.keras.utils.to_categorical(test_y, num_classes)

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    height_shift_range=0.1,
    width_shift_range=0.1,
    horizontal_flip=True,
    rescale=1./255.
)

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255.)

train_generator = train_datagen.flow(train_x, train_y_one, shuffle=True)
test_generator = test_datagen.flow(test_x, test_y_one)

In [0]:
model.fit_generator(
    train_generator, train_step, epochs,
    validation_data=test_generator, validation_steps=test_step
)

Epoch 1/100
1562/1562 [==============================] - 224s 144ms/step - loss: 1.7445 - acc: 0.3379 - val_loss: 1.9419 - val_acc: 0.3221
Epoch 2/100
1562/1562 [==============================] - 195s 125ms/step - loss: 1.2634 - acc: 0.5421 - val_loss: 1.8630 - val_acc: 0.4580
Epoch 3/100
1562/1562 [==============================] - 193s 124ms/step - loss: 1.0286 - acc: 0.6333 - val_loss: 1.6555 - val_acc: 0.5171
Epoch 4/100
1562/1562 [==============================] - 193s 124ms/step - loss: 0.8935 - acc: 0.6838 - val_loss: 1.0706 - val_acc: 0.6366
Epoch 5/100
1562/1562 [==============================] - 193s 124ms/step - loss: 0.7834 - acc: 0.7278 - val_loss: 0.9926 - val_acc: 0.6697
Epoch 6/100
1562/1562 [==============================] - 193s 123ms/step - loss: 0.6954 - acc: 0.7570 - val_loss: 1.1482 - val_acc: 0.6330
Epoch 7/100
1562/1562 [==============================] - 193s 124ms/step - loss: 0.6295 - acc: 0.7820 - val_loss: 1.2662 - val_acc: 0.5989
Epoch 8/100
1562/1562 [====

In [0]:
def lr_schedule(epoch):
  lr = 1e-3
  if epoch > 180:
      lr *= 0.5e-3
  elif epoch > 160:
      lr *= 1e-3
  elif epoch > 120:
      lr *= 1e-2
  elif epoch > 80:
      lr *= 1e-1
  print('Learning rate: ', lr)
  return lr

In [8]:
model.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(lr_schedule(0)),
    metrics=['accuracy']
)

Learning rate:  0.001


In [10]:
history = model.fit_generator(
    train_generator, train_step, epochs,
    validation_data=test_generator, validation_steps=test_step
)

Epoch 1/100
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
1562/1562 [==============================] - 1249s 800ms/step - loss: 1.3884 - acc: 0.4951 - val_loss: 1.5673 - val_acc: 0.5104
Epoch 2/100
1562/1562 [==============================] - 1237s 792ms/step - loss: 0.9714 - acc: 0.6549 - val_loss: 1.3369 - val_acc: 0.5972
Epoch 3/100
1562/1562 [==============================] - 1237s 792ms/step - loss: 0.7761 - acc: 0.7312 - val_loss: 0.8148 - val_acc: 0.7244
Epoch 4/100
1562/1562 [==============================] - 1240s 794ms/step - loss: 0.6547 - acc: 0.7737 - val_loss: 0.6993 - val_acc: 0.7497
Epoch 5/100
1562/1562 [==============================] - 1238s 793ms/step - loss: 0.5757 - acc: 0.8029 - val_loss: 0.7715 - val_acc: 0.7397
Epoch 6/100
1562/1562 [==============================] - 1240s 794ms/step - loss: 0.5189 - acc: 0.8221 - val_loss: 0.7493 - val_acc: 0.7535
Epoch 7/100
1562/1562 [==============================] - 1237s 792

KeyboardInterrupt: ignored